<a href="https://colab.research.google.com/github/aditi2259/AF-New/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import zipfile
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
zip_path = '/content/drive/MyDrive/ECG/training2017.zip'
extract_path = '/content/ECG/training2017'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [17]:
data = pd.read_csv('/content/drive/MyDrive/ECG/REFERENCE-v0.csv', header=None, names=['id', 'value'])
label_mapping = {label: idx for idx, label in enumerate(data['value'].unique())}
data['value'] = data['value'].map(label_mapping)

In [18]:
file_dir = extract_path  # Unzipped directory containing .mat files
file_paths = [os.path.join(file_dir, f"{row['id']}.mat") for _, row in data.iterrows()]
labels = data['value'].values

In [19]:
class Datagenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size, shuffle=True, target_shape=(9000, 1), num_class=len(label_mapping)):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_shape = target_shape
        self.num_class = num_class
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.file_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = to_categorical(batch_labels, num_classes=self.num_class)
        batch_data = [self.load_mat_file(path) for path in batch_paths]
        return tf.convert_to_tensor(batch_data), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            data = list(zip(self.file_paths, self.labels))
            np.random.shuffle(data)
            self.file_paths, self.labels = zip(*data)

    def load_mat_file(self, path):
        try:
            ecg_dict = loadmat(path)
            ecg_data = ecg_dict['val'].flatten()
            ecg_data = np.pad(ecg_data, (0, max(0, self.target_shape[0] - len(ecg_data))), 'constant')
            ecg_data = ecg_data[:self.target_shape[0]]
            ecg_data = ecg_data.reshape(self.target_shape)
            return ecg_data
        except FileNotFoundError:
            print(f"File not found: {path}")
            return np.zeros(self.target_shape)

In [20]:
batch_size = 32
train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

train_generator = Datagenerator(train_paths, train_labels, batch_size=batch_size, num_class=len(label_mapping))
val_generator = Datagenerator(val_paths, val_labels, batch_size=batch_size, num_class=len(label_mapping))



In [21]:
def CNN_model(input_shape):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

    model = Sequential()

    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))

    model.add(Dense(len(label_mapping), activation='softmax'))

    return model

In [ ]:
input_shape = (9000, 1)
model = CNN_model(input_shape)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

File not found: /content/ECG/training2017/A02203.mat
File not found: /content/ECG/training2017/A05924.mat
File not found: /content/ECG/training2017/A03245.mat
File not found: /content/ECG/training2017/A06649.mat
File not found: /content/ECG/training2017/A04824.mat
File not found: /content/ECG/training2017/A07865.mat
File not found: /content/ECG/training2017/A04795.mat
File not found: /content/ECG/training2017/A00392.mat
File not found: /content/ECG/training2017/A07069.mat
File not found: /content/ECG/training2017/A03094.mat
File not found: /content/ECG/training2017/A07947.mat
File not found: /content/ECG/training2017/A00528.mat
File not found: /content/ECG/training2017/A06703.mat
File not found: /content/ECG/training2017/A01371.mat
File not found: /content/ECG/training2017/A06490.mat
File not found: /content/ECG/training2017/A05571.mat
File not found: /content/ECG/training2017/A07628.mat
File not found: /content/ECG/training2017/A07077.mat
File not found: /content/ECG/training2017/A039